experimental + custom components

In [1]:
# 해당 셀을 실행한 후에 반드시 "런타임 다시시작"을 해주세요
!pip3 install kfp google-cloud-aiplatform --upgrade -q --user

     |████████████████████████████████| 298 kB 9.1 MB/s 
     |████████████████████████████████| 1.7 MB 60.8 MB/s 
     |████████████████████████████████| 127 kB 53.6 MB/s 
     |████████████████████████████████| 636 kB 49.4 MB/s 
     |████████████████████████████████| 106 kB 51.7 MB/s 
     |████████████████████████████████| 1.6 MB 58.6 MB/s 
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 56 kB 1.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 6.6 MB/s 
     |████████████████████████████████| 10.1 MB 40.8 MB/s 
     |████████████████████████████████| 93 kB 1.4 MB/s 
     |████████████████████████████████| 75 kB 3.7 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |██████████████

In [1]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component, ClassificationMetrics, Metrics)
from kfp.v2.google import experimental

from google.cloud import aiplatform as vertex_ai

import tensorflow as tf

import time
from typing import Optional

# 환경변수 설정
아래 항목들을 본인 환경에 맞게 수정해주세요.

* PROJECT_ID = <프로젝트 ID>
* REGION = <리전>
* BUCKET_NAME = <bucket 이름>
* USER = <user 이름>

In [2]:
# GCP 설정
PROJECT_ID = 'summarization-336407'
REGION = "us-central1"
USER = "JeongMin-Do"
BUCKET = "summarization-123423"
PIPELINE_NAME = "summarization"

# wandb 설정
# WANDB_API_KEY = ""

In [3]:
ts = int(time.time())
BUCKET_NAME = f"gs://{BUCKET}"
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/{USER}"
WORKING_DIR = f"{PIPELINE_ROOT}/{ts}"
MODEL_DISPLAY_NAME = f"train_deploy{ts}"
MODEL_DIR = f"pipeline_root/{USER}/{ts}"

## GCP 연결

In [5]:
# gcp 연결
# using credential file
from google.oauth2 import service_account

# credentials = service_account.Credentials.from_service_account_file(
#     '/content/vertexai-pipeline-328904-b011e928c725.json')
# vertex_ai.init(project=PROJECT_ID,
#                credentials=credentials
#                )


from google.colab import auth as google_auth

google_auth.authenticate_user()
vertex_ai.init(project=PROJECT_ID)

# Pipeline 정의

#### 해당 부분을 요구에 맞게 수정해주세요

In [6]:
@component
def train(training_state: str):
    print(f"training task: {training_state}")

@component(base_image="silverstar456/vertexai:aiplatform")
def model_upload(
    project: str,
    location: str,
    display_name: str,
    serving_container_image_uri: str,
    artifact_uri: Optional[str]=None,
):
    from google.cloud import aiplatform as vertex_ai

    vertex_ai.init(project=project, location=location)

    model = vertex_ai.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
    )

@component(base_image="silverstar456/vertexai:aiplatform")
def model_deploy(
    project: str,
    location: str,
    display_name: str,
):
    from google.cloud import aiplatform as vertex_ai

    vertex_ai.init(project=project, location=location)
    model = vertex_ai.Model.list()[0]

    endpoint = vertex_ai.Endpoint.create(display_name=display_name)
    model.deploy(endpoint=endpoint, 
                 machine_type="n1-standard-2", 
                 min_replica_count=1, 
                 max_replica_count=1)

In [12]:
@kfp.dsl.pipeline(name="summarization-" + str(ts), pipeline_root=PIPELINE_ROOT)
def pipeline():
    # model train
    train_task = train("model training")
    experimental.run_as_aiplatform_custom_job(
        train_task,
        worker_pool_specs=[
            {
                "containerSpec": {
                    "env": [{"name": "WORKING_DIR", "value": MODEL_DIR},
                            {"name": "BUCKET", "value": BUCKET}],
                    "imageUri": "silverstar456/vertexai:summarization", # 사전에 빌드한 도커 이미지 URI
                },
                "replicaCount": "1",
                "machineSpec": {
                    "machineType": "n1-standard-8", # 실행시 사용할 머신 타입 GCP 문서 참조
                    # "accelerator_type": vertex_ai.gapic.AcceleratorType.NVIDIA_TESLA_K80, # GPU 타입
                    # "accelerator_count": 1, # GPU 개수
                },
            }
        ],
    )

    # model upload
    uploader = model_upload(
        project=PROJECT_ID,
        location=REGION,
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=WORKING_DIR,
        serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest"
        )
    uploader.after(train_task)
    
    # model deploy
    deployer = model_deploy(
        project=PROJECT_ID,
        location=REGION,
        display_name=MODEL_DISPLAY_NAME,
    )
    deployer.after(uploader)

In [13]:
compiler.Compiler().compile(
    pipeline_func = pipeline, package_path = "summarization.json"
)

pipeline = vertex_ai.PipelineJob(
    display_name="summarization",
    template_path="summarization.json",
    parameter_values={},
)

pipeline.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


/root/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,
/root/.local/lib/python3.7/site-packages/kfp/v2/google/experimental/custom_job.py:78: FutureWarning: This function will be deprecated in v2.0.0
  'This function will be deprecated in v2.0.0', category=FutureWarning,
